# Devices

A significant aspect of working with PyTorch involves managing the devices on which your code runs. This page focuses on that topic.

**Note:** To fully grasp the concepts discussed here, you'll need a GPU that supports CUDA. Consider using Google Colab for this purpose.

In [2]:
import torch
from torch.utils import benchmark

To check the status of your GPU, use the `nvidia-smi` command as shown in the cell below.

In [2]:
!nvidia-smi

Wed Aug  7 11:41:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Selecting device

By default, all objects are created on the `cpu`. However, you can use the `to` method to move them to a different device.

---

By default, tensors created in PyTorch are allocated to the CPU device. The following code demonstrates this by creating a tensor and printing its device:

In [15]:
cpu_tensor = torch.ones([3, 3])
cpu_tensor.device

device(type='cpu')

The following code switches the device to cuda and displays the device:

In [16]:
cuda_tensor = cpu_tensor.to("cuda")
cuda_tensor.device

device(type='cuda', index=0)

Another option is to use `cuda` method:

In [18]:
cpu_tensor.cuda().device

device(type='cuda', index=0)

### Network

PyTorch networks have a `to` method that conveniently applies a specified device to all of the network's tensors.

---

In the following code cell, we create a network and then print the devices of its tensors.

In [21]:
network = torch.nn.Sequential(
    torch.nn.Linear(3,3),
    torch.nn.ReLU()
)

for param in network.parameters():
  print(param.device)

cpu
cpu


After applying the `.to('cuda')` method to the network, we obtain a new object that represents the same network but with all its tensors now residing on the GPU.

In [22]:
cuda_network = network.to('cuda')

for param in cuda_network.parameters():
  print(param.device)

cuda:0
cuda:0


## Benchmark

Let's compare the performance of a PyTorch network running on the device (likely a GPU) and on the CPU using `torch.utils.benchmark`.

---

This code creates a large, inefficient neural network and attempts to use it with a large matrix, resulting in slow execution time.

In [5]:
dimentionality = 1000
network = torch.nn.Sequential(*[
    torch.nn.Linear(dimentionality, dimentionality) for i in range(100)
])
X = torch.randn([dimentionality, dimentionality])

benchmark.Timer(
    stmt="network(X)",
    globals={"network": network, "X": X}
).timeit(5)

network(X)
  1.73 s
  1 measurement, 5 runs , 1 thread

Now, we'll benchmark the same network but this time with its computations moved to the GPU to see how much faster it performs.

In [6]:
network = network.to('cuda')
X = X.to('cuda')

benchmark.Timer(
    stmt="network(X)", globals={"network": network, "X": X}
).timeit(5)

network(X)
  65.91 ms
  1 measurement, 5 runs , 1 thread